# *Computing the PFSS model*
***

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sunpy

In [ ]:
import pysmsh
import cider

### Load magnetogram & process

Load a HMI synchronic magnetogram

In [ ]:
import cider.magnetogram.hmi

In [ ]:
magnetogram_file = "../data/hmi.mrdailysynframe_polfil_720s.20211009_120000_TAI.Mr_polfil.fits"

In [ ]:
raw_magnetogram = cider.magnetogram.hmi.read_hmi_daily_synframe(magnetogram_file)

Remap to a uniform lon-lat grid which is required by the PFSS solver

In [ ]:
import cider.utils.map
import cider.magnetogram.flux

In [ ]:
# Create an empty map with the requested resolution
uniform_map \
    = cider.utils.map.create_full_sun_plate_carree_map(raw_magnetogram,
                                                       deg_per_pixel=1.0,
                                                       frame=raw_magnetogram.coordinate_frame.name)

In [ ]:
remapped_magnetogram = cider.utils.map.regrid_to_grid_of_map(raw_magnetogram, uniform_map)

Balance the magnetogram

In [ ]:
balanced_magnetogram = cider.magnetogram.flux.Balance().multiplicative(remapped_magnetogram)

In [ ]:
balanced_magnetogram.peek(vmin=-50, vmax=50)

In [ ]:
print(cider.magnetogram.flux.Flux(remapped_magnetogram).signed)
print(cider.magnetogram.flux.Flux(remapped_magnetogram).unsigned)

In [ ]:
print(cider.magnetogram.flux.Flux(balanced_magnetogram).signed)
print(cider.magnetogram.flux.Flux(balanced_magnetogram).unsigned)

### Compute the PFSS model

In [ ]:
import cider.models.pfss

Specify the radial grid coordinates. For the PFSS model, the outer radius defines the source surface

In [ ]:
r = np.linspace(1.0, 2.5, 256)*sunpy.sun.constants.radius.value

In [ ]:
# Instantiate the model
pfss = cider.models.pfss.PotentialFieldSourceSurfaceModel(balanced_magnetogram, r)

In [ ]:
# Compute the solution
pfss.compute()

In [ ]:
# Compute the magnetic field
magnetic_field = pfss.magnetic_field()

Do some additional analysis: compute the divergence of the magnetic field

In [ ]:
import pysmsh.difference.staggered_curvilinear as curvilinear

In [ ]:
div_magnetic_field = pysmsh.Field.Scalar(pfss.mesh, "cell_centered")

In [ ]:
curvilinear.divergence(magnetic_field, div_magnetic_field, curvilinear.SphericalGeometry())

Output the cell-centered magnetic field components for external visualization

In [ ]:
import pyevtk.hl as evtk

In [ ]:
B = magnetic_field

evtk.gridToVTK("pfss",
               B.mesh.edges.r/sunpy.sun.constants.radius.value,
               B.mesh.edges.clt,
               B.mesh.edges.lon,
               cellData={"Br" : 0.5*(B.data[0][1::, :, :] + B.data[0][0:-1, :, :]),
                         "Bt" : 0.5*(B.data[1][:, 1::, :] + B.data[1][:, 0:-1, :]),
                         "Bp" : 0.5*(B.data[2][:, :, 1::] + B.data[2][:, :, 0:-1]),
                         "divB" : div_magnetic_field.data[:, :, :]
                        }
              )